In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#Pesaje Recibido en Puntos Verdes (2015-2016) en kilogramos.
#Importamos
filename = r'C:\Users\leoma\Desktop\UTN\Ciencia de Datos\DataSets\TP2019\pesaje-recibido-en-puntos-verdes-2015-2016.csv'
pesajes = pd.read_csv(filename, delimiter=';', decimal=',')
print(f'Pesajes tiene: {np.shape(pesajes)[0]} filas y {np.shape(pesajes)[0]} columnas')

Pesajes tiene: 3080 filas y 3080 columnas


###  Cuantos puntos y comunas distintas hay?

Mapa de comunas en este [link](https://www.google.com/imgres?imgurl=https%3A%2F%2Fupload.wikimedia.org%2Fwikipedia%2Fcommons%2Fthumb%2F4%2F4b%2FBuenos_Aires_Comunas.svg%2F1200px-Buenos_Aires_Comunas.svg.png&imgrefurl=https%3A%2F%2Fes.wikipedia.org%2Fwiki%2FComunas_de_la_ciudad_de_Buenos_Aires&docid=Ezlwe5ezJGTRlM&tbnid=FSUavRrKMRElQM%3A&vet=10ahUKEwiXpNLmjs7kAhW2HbkGHTRqD2sQMwhRKAAwAA..i&w=1200&h=1222&bih=878&biw=958&q=comunas%20caba&ved=0ahUKEwiXpNLmjs7kAhW2HbkGHTRqD2sQMwhRKAAwAA&iact=mrc&uact=8)

In [3]:
comunas_distintas = np.unique(pesajes.COMUNA)
print(f'Hay {len(comunas_distintas)} comunas distintas')

puntos_distintos = np.unique(pesajes.PUNTO)
print(f'Hay {len(puntos_distintos)} puntos distintos distribuidos entre las comunas')
#pesajes['VIDRIO'][0]

Hay 15 comunas distintas
Hay 32 puntos distintos distribuidos entre las comunas


### Transformamos los meses a valores numéricos

In [4]:
pesajes.loc[ pesajes['MES'] == "ENERO", 'MES'] = 1
pesajes.loc[ pesajes['MES'] == "FEBRERO", 'MES'] = 2
pesajes.loc[ pesajes['MES'] == "MARZO", 'MES'] = 3
pesajes.loc[ pesajes['MES'] == "ABRIL", 'MES'] = 4
pesajes.loc[ pesajes['MES'] == "MAYO", 'MES'] = 5
pesajes.loc[ pesajes['MES'] == "JUNIO", 'MES'] = 6
pesajes.loc[ pesajes['MES'] == "JULIO", 'MES'] = 7
pesajes.loc[ pesajes['MES'] == "AGOSTO", 'MES'] = 8
pesajes.loc[ pesajes['MES'] == "SEPTIEMBRE", 'MES'] = 9
pesajes.loc[ pesajes['MES'] == "OCTUBRE", 'MES'] = 10
pesajes.loc[ pesajes['MES'] == "NOVIEMBRE", 'MES'] = 11
pesajes.loc[ pesajes['MES'] == "DICIEMBRE", 'MES'] = 12

#Separo 2015 y 2016
pesajes_2015 = pesajes.loc[pesajes['ANIO'] == 2015]
pesajes_2016 = pesajes.loc[pesajes['ANIO'] == 2016]


##   Criterios elegidos para limpiar el Dataframe

### Semana
Si una semana esta llena de 0s, "", " ", NaNs es considerada VACIA
### Mes
Si un mes tiene 3 o más semanas VACIAS -> Esta VACIO
Si un mes no esta cargado en el dataframe -> Esta VACIO
### Punto
Si un Punto tiene 4 o mas meses VACIOS en 2015 y 2016 -> Esta VACIO



### Cuento los meses que aparecen tienen 3 o mas semanas vacias, es decir con su fila rellena de  0, "", " " o NaN.

In [5]:
#Primero filtro las que tienen meses con todas las filas llenas de cero (si tienen esas 3 vacias, el resto tambien)
filtro1 = pesajes_2015['VIDRIO'].isin([0,np.nan," "])
filtro2 = pesajes_2015['PAPEL_Y_CARTON'].isin([0,np.nan," "])
filtro3 = pesajes_2015['AVUS'].isin([0,np.nan," "])
sin_vacio = (filtro1 & filtro2 & filtro3)
pesajes_2015_semanas_vacias = pesajes_2015[sin_vacio].drop_duplicates().sort_values(by=['PUNTO', 'MES','SEMANA']).copy().reset_index(drop=True)


filtro1 = pesajes_2016['VIDRIO'].isin([0,np.nan," "])
filtro2 = pesajes_2016['PAPEL_Y_CARTON'].isin([0,np.nan," "])
filtro3 = pesajes_2016['AVUS'].isin([0,np.nan," "])
sin_vacio = (filtro1 & filtro2 & filtro3)
pesajes_2016_semanas_vacias = pesajes_2016[sin_vacio].drop_duplicates().sort_values(by=['PUNTO', 'MES','SEMANA']).copy().reset_index(drop=True)

In [6]:
# Leo todas las semanas faltantes, si llego a 3 en el mismo mes lo guardo en el
# siguiente dataframe para luego borrarlos todos juntos
meses_Vacios_2015 = pd.DataFrame({"PUNTO":[],"ANIO":[],"MES":[]})

mesAnterior = 0 #lo arranco en un mes inexistente

for indice in range(0, len(pesajes_2015_semanas_vacias)):
    mesActual = pesajes_2015_semanas_vacias.iloc[indice].MES
    if (mesAnterior == mesActual):
        semanas_vacias_del_Mes += 1
        if(semanas_vacias_del_Mes != 3):
            continue
        else:
            meses_Vacios_2015 = meses_Vacios_2015.append(
                {"PUNTO": pesajes_2015_semanas_vacias.iloc[indice].PUNTO,
                 "ANIO": 2015,
                 "MES": mesActual}, ignore_index=True)
    else:
        mesAnterior = mesActual
        semanas_vacias_del_Mes = 1 #Reseteo el contador

meses_Vacios_2015_tabla = meses_Vacios_2015.groupby(["PUNTO", "ANIO"])["MES"].count().sort_values(ascending=False)
meses_Vacios_2015_tabla = pd.concat([meses_Vacios_2015_tabla], axis=1, keys=['Meses vacios por tener al menos 3 semanas vacias'])
meses_Vacios_2015_tabla

,,Meses vacios por tener al menos 3 semanas vacias
PUNTO,ANIO,
PLAZA 25 DE AGOSTO,2015.0,11
PLAZA VELAZCO IBARRA,2015.0,4
ESTACION VILLA SOLDATI,2015.0,4


In [7]:
# Leo todas las semanas faltantes, si llego a 3 en el mismo mes lo guardo en el
# siguiente dataframe para luego borrarlos todos juntos
meses_Vacios_2016 = pd.DataFrame({"PUNTO":[],"ANIO":[],"MES":[]})

mesAnterior = 0 #lo arranco en un mes inexistente

for indice in range(0, len(pesajes_2016_semanas_vacias)):
    mesActual = pesajes_2016_semanas_vacias.iloc[indice].MES
    if (mesAnterior == mesActual):
        semanas_vacias_del_Mes += 1
        if(semanas_vacias_del_Mes != 3):
            continue
        else:
            meses_Vacios_2016 = meses_Vacios_2016.append(
                {"PUNTO": pesajes_2016_semanas_vacias.iloc[indice].PUNTO,
                 "ANIO": 2016,
                 "MES": mesActual}, ignore_index=True)
    else:
        mesAnterior = mesActual
        semanas_vacias_del_Mes = 1 #Reseteo el contador

meses_Vacios_2016_tabla = meses_Vacios_2016.groupby(["PUNTO", "ANIO"])["MES"].count().sort_values(ascending=False)
meses_Vacios_2016_tabla = pd.concat([meses_Vacios_2016_tabla], axis=1, keys=['Meses vacios por tener al menos 3 semanas vacias'])
meses_Vacios_2016_tabla

,,Meses vacios por tener al menos 3 semanas vacias
PUNTO,ANIO,
PLAZA 25 DE AGOSTO,2016.0,8
ESTACION VILLA SOLDATI,2016.0,8
PLAZA SAN MARTIN,2016.0,3
PLAZA M BELGRANO,2016.0,2
PLAZA SUDAMERICANA,2016.0,1


2015 no tiene meses faltantes no cargados.

En el Año 2016 hay muchos meses que no estan cargados. Vamos a contarlos incluyendo los que estan vacíos por cada uno de los Puntos.

Luego eliminamos los barrios que tengan 4 o mas meses vacios entre los 2 años.

In [8]:
## Creo un df de los meses que tengo cargados de cada año, ordenados
meses_cargados_2015 = pesajes_2015.iloc[:,[0,3]].drop_duplicates().sort_values(by=['PUNTO', 'MES'])
meses_cargados_2015 = meses_cargados_2015.copy().reset_index(drop=True)
print(f'\n2015 tiene todos los meses cargados, 32(puntos)x12(meses) = {np.shape(meses_cargados_2015)[0]}\n')

meses_cargados_2016 = pesajes_2016.iloc[:,[0,3]].drop_duplicates().sort_values(by=['PUNTO', 'MES'])
meses_cargados_2016 = meses_cargados_2016.copy().reset_index(drop=True)
print(f'2016 le faltan meses en algunos Puntos, tiene {np.shape(meses_cargados_2016)[0]} meses cargados')
print(f'\nLos numeros de meses que faltan por cada Punto esta guardada en\nla variable meses_faltantes')
    
## Los comparo y obtengo un df de todos los meses faltantes con PUNTO y MES
meses_faltantes = pd.merge(meses_cargados_2015, meses_cargados_2016,
                           how='outer', indicator=True).query('_merge == "left_only"')

meses_faltantes_por_barrio = meses_faltantes.groupby(["PUNTO"])["MES"].count().sort_values(ascending=False)
    
## Agrego los 2 puntos sin falta de meses para que quede completa la lista
puntos_sin_meses_faltantes = list(filter(lambda x: x not in meses_faltantes_por_barrio.index, puntos_distintos))
meses_faltantes_por_barrio[puntos_sin_meses_faltantes[0]] = 0
meses_faltantes_por_barrio[puntos_sin_meses_faltantes[1]] = 0

Tabla_Faltantes = pd.concat([meses_faltantes_por_barrio], axis=1, keys=['Meses faltantes por barrio en 2016'])

meses_faltantes = meses_faltantes.iloc[:,0:2]

#Tabla_Faltantes

print("\nTabla_Faltantes tiene la cantidad de meses faltantes por barrio\nen 2016")


2015 tiene todos los meses cargados, 32(puntos)x12(meses) = 384

2016 le faltan meses en algunos Puntos, tiene 298 meses cargados

Los numeros de meses que faltan por cada Punto esta guardada en
la variable meses_faltantes

Tabla_Faltantes tiene la cantidad de meses faltantes por barrio
en 2016


In [9]:
Tabla_de_Vacios_Completa = meses_faltantes.groupby(["PUNTO"])["MES"].count().sort_values(ascending=False)

## Agrego los 2 puntos sin falta de meses para que quede completa la lista
puntos_sin_meses_faltantes_2 = list(filter(lambda x: x not in Tabla_de_Vacios_Completa.index, puntos_distintos))
Tabla_de_Vacios_Completa[puntos_sin_meses_faltantes_2[0]] = 0
Tabla_de_Vacios_Completa[puntos_sin_meses_faltantes_2[1]] = 0
Tabla_de_Vacios_Completa = Tabla_de_Vacios_Completa.to_frame().rename(columns = {"MES":"Faltantes_2016"})

In [10]:
Agregar_a_tabla_vacias = pesajes_2016_semanas_vacias.groupby(["PUNTO", "MES"]).filter(lambda x: x["SEMANA"].count() >= 3)
Agregar_a_tabla_vacias_filtrada_2016 = Agregar_a_tabla_vacias.iloc[:,[0,3]].drop_duplicates().sort_values(by=['PUNTO', 'MES'])
Agregar_a_tabla_vacias_filtrada_2016 = Agregar_a_tabla_vacias_filtrada_2016.groupby(["PUNTO"]).size().to_frame().rename(columns = {0: "Vacios_2016"})

Agregar_a_tabla_vacias_2015 = pesajes_2015_semanas_vacias.groupby(["PUNTO", "MES"]).filter(lambda x: x["SEMANA"].count() >= 3)
Agregar_a_tabla_vacias_filtrada_2015 = Agregar_a_tabla_vacias_2015.iloc[:,[0,3]].drop_duplicates().sort_values(by=['PUNTO', 'MES'])
Agregar_a_tabla_vacias_filtrada_2015 = Agregar_a_tabla_vacias_filtrada_2015.groupby(["PUNTO"]).size().to_frame().rename(columns = {0: "Vacios_2015"})

Tabla_de_Vacios_Completa = pd.merge(Tabla_de_Vacios_Completa, Agregar_a_tabla_vacias_filtrada_2016, how='left', on='PUNTO').fillna(0)
Tabla_de_Vacios_Completa = pd.merge(Tabla_de_Vacios_Completa, Agregar_a_tabla_vacias_filtrada_2015, how='left', on='PUNTO').fillna(0)
Tabla_de_Vacios_Completa['Meses_No_Utilizables'] = Tabla_de_Vacios_Completa.Faltantes_2016 + Tabla_de_Vacios_Completa.Vacios_2016 + Tabla_de_Vacios_Completa.Vacios_2015

# Muestro la tabla ordenada por Meses No utilizables
Tabla_de_Vacios_Completa.sort_values(by=['Meses_No_Utilizables'],ascending=False)

,Faltantes_2016,Vacios_2016,Vacios_2015,Meses_No_Utilizables
PUNTO,,,,
PLAZA 25 DE AGOSTO,4,8.0,11.0,23.0
ESTACION VILLA SOLDATI,4,8.0,4.0,16.0
PLAZA VELAZCO IBARRA,3,0.0,4.0,7.0
PLAZA M BELGRANO,3,2.0,0.0,5.0
PLAZA SAN MARTIN,1,3.0,0.0,4.0
PUERTO MADERO,3,0.0,0.0,3.0
PLAZA CASTELLI (COMUNA 13),3,0.0,0.0,3.0
PLAZA VERA PENIALOZA,3,0.0,0.0,3.0
PLAZA VELEZ SARSFIELD,3,0.0,0.0,3.0


### Decidimos borrar los siguientes 5 de 32 Puntos por tener tantos datos faltantes según el criterio explicado anteriormente
    PLAZA 25 DE AGOSTO
    ESTACION VILLA SOLDATI
    PLAZA VELAZCO IBARRA
    PLAZA M BELGRANO
    PLAZA SAN MARTIN

In [22]:
pesajes_criterio_de_limpieza = pesajes.loc[~pesajes['PUNTO'].isin(['ESTACION VILLA SOLDATI',
                                                                 'PLAZA 25 DE AGOSTO',
                                                                 'PLAZA VELAZCO IBARRA',
                                                                 'PLAZA M BELGRANO',
                                                                 'PLAZA SAN MARTIN'])]
pesajes_criterio_de_limpieza

,PUNTO,COMUNA,ANIO,MES,SEMANA,VIDRIO,PAPEL_Y_CARTON,METAL,PLASTICO_PET,TELGOPOR,PLASTICOS_TODOS_LOS_DEMAS,TETRA_BRICK,PEQUENIOS_ELECTRODOMESTICOS,APARATOS_ELECTRONICOS_EN_DESUSO,AVUS
90,PARQUE AVELLANEDA,COMUNA 9,2015,4,1,"115,01","166,29","16,81",,"1,17","81,31","4,95",,,
91,PARQUE AVELLANEDA,COMUNA 9,2015,4,2,"113,971","113,558","28,473",,"1,309","90,877","6,309",,,
92,PARQUE AVELLANEDA,COMUNA 9,2015,4,3,"140,212","152,93","17,685",,"2,272","97,397","6,942",,,
93,PARQUE AVELLANEDA,COMUNA 9,2015,4,4,"88,981","129,333","14,799",,"1,313","68,857","4,322",,,
94,PARQUE AVELLANEDA,COMUNA 9,2015,4,5,"52,79","84,22","3,51",,0,"31,27",0,,,
95,PARQUE AVELLANEDA,COMUNA 9,2015,8,1,"118,564","233,942","17,714","162,621",0,"93,361",4,,,
96,PARQUE AVELLANEDA,COMUNA 9,2015,8,2,"87,137","140,941","23,28","164,212",0,"56,724","0,735",,,
97,PARQUE AVELLANEDA,COMUNA 9,2015,8,3,"143,021","251,001","31,318","214,616",0,"110,665","9,273",,,
98,PARQUE AVELLANEDA,COMUNA 9,2015,8,4,"161,577","284,751","30,039","199,761",0,"115,304","6,4",,,
99,PARQUE AVELLANEDA,COMUNA 9,2015,8,5,"174,18","363,12","16,808","181,444",0,"100,712","5,514",,,
